In [80]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

# Display settings
pd.set_option("display.max_columns", None)

# File path (update if needed)
file_path = "Dataset\Healthcare Insurance Claims Management Dataset.xlsx"

# Load workbook
xls = pd.ExcelFile(file_path)

# Load all sheets into a dictionary
dfs = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in xls.sheet_names}

dfs.keys()


dict_keys(['Claims_Fact_raw', 'Provider_Master_raw', 'Member_Master_raw', 'Policy_Master_raw', 'Procedure_Master', 'Diagnosis_Master_ICD'])

In [81]:

for name, df in dfs.items():
    print(f"\n===== {name} =====")
    print("Rows:", len(df))
    print("Columns:", df.columns.tolist())



===== Claims_Fact_raw =====
Rows: 12000
Columns: ['ClaimID', 'MemberID', 'PolicyID', 'ProviderID', 'ClaimType', 'ClaimSubmissionDate', 'AdmissionDate', 'DischargeDate', 'ClaimedAmount', 'ApprovedAmount', 'ICDCode(s)', 'ProcedureCode(s)', 'PreAuthNumber', 'PreAuthStatus', 'QueryRaiseDate', 'SettlementDate', 'TPANotes', 'Currency', 'NetworkStatus']

===== Provider_Master_raw =====
Rows: 350
Columns: ['ProviderID', 'ProviderName', 'Location', 'City', 'State', 'NetworkType', 'PackageRates']

===== Member_Master_raw =====
Rows: 9000
Columns: ['MemberID', 'AgeBand', 'Gender', 'City', 'State', 'PolicyID']

===== Policy_Master_raw =====
Rows: 3000
Columns: ['PolicyID', 'PolicyType', 'CoverageLimit', 'StartDate', 'EndDate', 'PremiumAmount']

===== Procedure_Master =====
Rows: 120
Columns: ['ProcedureCode', 'ProcedureCategory', 'StandardRate', 'LOSStandard']

===== Diagnosis_Master_ICD =====
Rows: 60
Columns: ['ICDCode', 'DiagnosisCategory', 'ValidProcedureCategories']


In [82]:
claims = dfs['Claims_Fact_raw']
providers = dfs['Provider_Master_raw']
members = dfs['Member_Master_raw']
policies = dfs['Policy_Master_raw']
procedures = dfs['Procedure_Master']
diagnosis = dfs['Diagnosis_Master_ICD']

In [83]:
audit_log = []

def log_issue(table, column, issue_type, rule, count, sample=None):
    audit_log.append({
        "table": table,
        "column": column,
        "issue_type": issue_type,
        "rule": rule,
        "rows_affected": count,
        "sample_values": sample
    })


In [84]:
# ----------------------------------------------
# 5. Primary Key Duplicate Checks
# ----------------------------------------------

# Claims_Fact: ClaimID
dups = claims[claims.duplicated(subset=['ClaimID'], keep=False)]
log_issue("Claims_Fact", "ClaimID", "Duplicate Key", "ClaimID must be unique", 
          len(dups), dups['ClaimID'].head().tolist())

# Provider
dups = providers[providers.duplicated(subset=['ProviderID'], keep=False)]
log_issue("Provider_Master", "ProviderID", "Duplicate Key", "ProviderID must be unique", 
          len(dups),dups['ProviderID'].head().tolist())

# Member
dups = members[members.duplicated(subset=['MemberID'], keep=False)]
log_issue("Member_Master", "MemberID", "Duplicate Key", "MemberID must be unique", 
          len(dups),dups['MemberID'].head().tolist())

# Policy
dups = policies[policies.duplicated(subset=['PolicyID'], keep=False)]
log_issue("Policy_Master", "PolicyID", "Duplicate Key", "PolicyID must be unique", 
          len(dups),dups['PolicyID'].head().tolist())


In [85]:
# ----------------------------------------------
# 6. Normalize Dates
# ----------------------------------------------
date_cols = [
    'ClaimSubmissionDate','AdmissionDate','DischargeDate','QueryRaiseDate','SettlementDate'
]

for col in date_cols:
    if col in claims.columns:
        claims[col] = pd.to_datetime(claims[col], errors='coerce')


# Date Logic Audit

In [86]:
invalid_stay = claims[claims['DischargeDate'] < claims['AdmissionDate']]
log_issue("Claims_Fact", "DischargeDate", "Invalid Date Order",
          "DischargeDate < AdmissionDate", len(invalid_stay),
          invalid_stay[['ClaimID','AdmissionDate','DischargeDate']].head().to_dict())


In [87]:
invalid_cycle = claims[claims['SettlementDate'] < claims['ClaimSubmissionDate']]
log_issue("Claims_Fact", "SettlementDate", "Invalid Date Order",
          "SettlementDate < ClaimSubmissionDate", len(invalid_cycle),invalid_cycle[['ClaimID','SettlementDate','ClaimSubmissionDate']].head().to_dict())


In [88]:
missing_discharge = claims[claims['DischargeDate'].isna()]
log_issue("Claims_Fact", "DischargeDate", "Missing", "DischargeDate is NULL",
          len(missing_discharge),missing_discharge['ClaimID'].head().to_dict())

missing_settlement = claims[claims['SettlementDate'].isna()]
log_issue("Claims_Fact", "SettlementDate", "Missing", "SettlementDate is NULL",
          len(missing_settlement),missing_settlement['ClaimID'].head().to_dict())


In [89]:
invalid_amounts = claims[claims['ApprovedAmount'] > claims['ClaimedAmount']]
log_issue("Claims_Fact", "ApprovedAmount", "Invalid Amount",
          "ApprovedAmount > ClaimedAmount", len(invalid_amounts),invalid_amounts[['ClaimID','ApprovedAmount','ClaimedAmount']].head().to_dict())


In [90]:
neg_amount = claims[claims['ClaimedAmount'] < 0]
log_issue("Claims_Fact", "ClaimedAmount", "Negative Amount",
          "ClaimedAmount < 0", len(neg_amount),neg_amount['ClaimedAmount'].head().to_dict())


# Outlier Detection (Z-Score)

In [91]:
#Merge procedure categories first
claims = claims.merge(
    procedures[['ProcedureCode','ProcedureCategory','StandardRate']],
    left_on='ProcedureCode(s)', right_on='ProcedureCode',
    how='left'
)


In [92]:
claims['z_score_claim'] = claims.groupby('ProcedureCategory')['ClaimedAmount']\
                                .transform(lambda x: (x - x.mean())/x.std(ddof=0))

outliers = claims[claims['z_score_claim'].abs() >= 3]

log_issue("Claims_Fact", "ClaimedAmount", "Outlier",
          "Z-score >= 3", len(outliers))


In [93]:
# ----------------------------------------------
# Validate ICD Codes (multi-code fields)
# ----------------------------------------------

# Step 1: Split ICDCode(s) into list
claims['ICD_list'] = claims['ICDCode(s)'].astype(str).str.split('|')

# Step 2: Explode list into separate rows
claims_exploded = claims.explode('ICD_list')

# Step 3: Validate each ICD code
invalid_icd_rows = claims_exploded[
    ~claims_exploded['ICD_list'].isin(diagnosis['ICDCode'])
]

# Step 4: Group by ClaimID to list invalid codes
invalid_by_claim = invalid_icd_rows.groupby('ClaimID')['ICD_list'].apply(list)

# Step 5: Log issue
log_issue(
    "Claims_Fact",
    "ICDCode(s)",
    "Invalid ICD Code",
    "One or more ICDs do not exist in Diagnosis_Master",
    len(invalid_by_claim),
    invalid_by_claim.head().to_dict()
)

invalid_by_claim.head()


Series([], Name: ICD_list, dtype: object)

In [94]:
# ----------------------------------------------------
# Validate Procedure Codes (supports multi-code fields)
# ----------------------------------------------------

# Step 1: Split ProcedureCode(s) into list
claims['Procedure_list'] = claims['ProcedureCode(s)'].astype(str).str.split('|')

# Step 2: Explode list into separate rows
claims_proc_exploded = claims.explode('Procedure_list')

# Step 3: Validate each procedure code against master
invalid_proc_rows = claims_proc_exploded[
    ~claims_proc_exploded['Procedure_list'].isin(procedures['ProcedureCode'])
]

# Step 4: Combine invalid codes per ClaimID
invalid_proc_by_claim = invalid_proc_rows.groupby('ClaimID')['Procedure_list'].apply(list)

# Step 5: Log issue
log_issue(
    "Claims_Fact",
    "ProcedureCode(s)",
    "Invalid Procedure Code",
    "One or more Procedure Codes do not exist in Procedure_Master",
    len(invalid_proc_by_claim),
    invalid_proc_by_claim.head().to_dict()
)

invalid_proc_by_claim.head()


Series([], Name: Procedure_list, dtype: object)

In [95]:
policy_check = claims.merge(
    policies[['PolicyID','StartDate','EndDate']],
    on='PolicyID',
    how='left'
)

invalid_policy = policy_check[
    ((policy_check['ClaimSubmissionDate'] < policy_check['StartDate']) |
      (policy_check['ClaimSubmissionDate'] > policy_check['EndDate']))
]

log_issue("Claims_Fact", "PolicyID", "Invalid Coverage",
          "Admission date outside policy period", len(invalid_policy),invalid_policy[['ClaimSubmissionDate','StartDate','EndDate']].head().to_dict())


In [96]:
len(invalid_policy)

9435

In [97]:
policy_check = claims.merge(
    policies[['PolicyID','StartDate','EndDate']],
    on='PolicyID',
    how='left'
)

valid_policy = policy_check[
    ((policy_check['ClaimSubmissionDate'] >= policy_check['StartDate']) &
      (policy_check['ClaimSubmissionDate'] <= policy_check['EndDate']))
]

log_issue("Claims_Fact", "PolicyID", "Invalid Coverage",
          "Admission date outside policy period", len(valid_policy),valid_policy[['ClaimSubmissionDate','StartDate','EndDate']].head().to_dict())

# Package Rate Cross-Checks

In [98]:
claims_pkg = claims.merge(
    providers[['ProviderID', 'NetworkType', 'PackageRates']],
    on='ProviderID',
    how='left'
)

In [99]:
claims['Procedure_list'] = claims['ProcedureCode(s)'].astype(str).str.split('|')

claims_proc_exp = claims.explode('Procedure_list')

claims_proc_exp = claims_proc_exp.merge(
    procedures[['ProcedureCode', 'StandardRate']],
    left_on='Procedure_list',
    right_on='ProcedureCode',
    how='left'
)
claims_proc_exp


,ClaimID,MemberID,PolicyID,ProviderID,ClaimType,ClaimSubmissionDate,AdmissionDate,DischargeDate,ClaimedAmount,ApprovedAmount,ICDCode(s),ProcedureCode(s),PreAuthNumber,PreAuthStatus,QueryRaiseDate,SettlementDate,TPANotes,Currency,NetworkStatus,ProcedureCode_x,ProcedureCategory,StandardRate_x,z_score_claim,ICD_list,Procedure_list,ProcedureCode_y,StandardRate_y
0,CLM000001,MBR000418,POL001439,PRV000064,OPD,2023-08-07,2023-08-02,2023-08-02,11032.91,9379.54,B20B|J18V,PRC0014,NaN,Not Required,NaT,NaT,Deductibles applied.,INR,In-Network,PRC0014,Pharmacy,9580.69,1.688783,"[B20B, J18V]",PRC0014,PRC0014,9580.69
1,CLM000002,MBR001547,POL000009,PRV000281,Hospitalization,2024-12-12,2024-12-02,2024-12-02,3585.74,2825.01,A09T,PRC0090|PRC0025,NaN,Not Required,NaT,2024-12-26,Room rent exceeded eligibility.,INR,Out-of-Network,NaN,NaN,NaN,NaN,[A09T],PRC0090,PRC0090,1530.24
2,CLM000002,MBR001547,POL000009,PRV000281,Hospitalization,2024-12-12,2024-12-02,2024-12-02,3585.74,2825.01,A09T,PRC0090|PRC0025,NaN,Not Required,NaT,2024-12-26,Room rent exceeded eligibility.,INR,Out-of-Network,NaN,NaN,NaN,NaN,[A09T],PRC0025,PRC0025,1754.04
3,CLM000003,MBR002778,POL002455,PRV000083,Hospitalization,2025-06-21,2025-06-20,2025-06-23,35010.21,29763.68,C50,PRC0102|PRC0017|PRC0074,PA-654823,Approved,NaT,NaT,Claim under medical review.,INR,In-Network,NaN,NaN,NaN,NaN,[C50],PRC0102,PRC0102,16410.67
4,CLM000003,MBR002778,POL002455,PRV000083,Hospitalization,2025-06-21,2025-06-20,2025-06-23,35010.21,29763.68,C50,PRC0102|PRC0017|PRC0074,PA-654823,Approved,NaT,NaT,Claim under medical review.,INR,In-Network,NaN,NaN,NaN,NaN,[C50],PRC0017,PRC0017,12867.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23969,CLM011999,MBR008814,POL001365,PRV000048,Hospitalization,2025-09-27,2025-09-26,2025-09-26,5021.27,4222.51,M17K,PRC0061,PA-148798,Approved,NaT,2025-10-27,Investigation initiated for high-cost claim.,USD,In-Network,PRC0061,Diagnostics,4792.17,1.265858,[M17K],PRC0061,PRC0061,4792.17
23970,CLM012000,MBR005075,POL000130,PRV000247,Maternity,2023-10-11,2023-10-11,2023-10-14,34593.01,29326.80,C50,PRC0038|PRC0097|PRC0022|PRC0036,PA-337705,Approved,NaT,2023-11-10,Pre-auth approved as per policy.,INR,In-Network,NaN,NaN,NaN,NaN,[C50],PRC0038,PRC0038,4122.71
23971,CLM012000,MBR005075,POL000130,PRV000247,Maternity,2023-10-11,2023-10-11,2023-10-14,34593.01,29326.80,C50,PRC0038|PRC0097|PRC0022|PRC0036,PA-337705,Approved,NaT,2023-11-10,Pre-auth approved as per policy.,INR,In-Network,NaN,NaN,NaN,NaN,[C50],PRC0097,PRC0097,1124.39
23972,CLM012000,MBR005075,POL000130,PRV000247,Maternity,2023-10-11,2023-10-11,2023-10-14,34593.01,29326.80,C50,PRC0038|PRC0097|PRC0022|PRC0036,PA-337705,Approved,NaT,2023-11-10,Pre-auth approved as per policy.,INR,In-Network,NaN,NaN,NaN,NaN,[C50],PRC0022,PRC0022,12392.34


In [78]:
standard_sum = claims_proc_exp.groupby('ClaimID')['StandardRate_y'].sum().reset_index()
standard_sum = standard_sum.rename(columns={"StandardRate_y": "TotalStandardRate"})

claims_std = claims.merge(standard_sum, on='ClaimID', how='left')

claims_std['min_allowed'] = claims_std['TotalStandardRate'] * 0.9
claims_std['max_allowed'] = claims_std['TotalStandardRate'] * 1.1


package_mismatch = claims_std[
    (claims_std['TotalStandardRate'].notna()) &
    (
        (claims_std['ClaimedAmount'] < claims_std['min_allowed']) |
        (claims_std['ClaimedAmount'] > claims_std['max_allowed'])
    )
]


log_issue(
    "Claims_Fact",
    "ClaimedAmount",
    "StandardRate Package Mismatch",
    "ClaimedAmount not within ±10% of total StandardRate for procedures",
    len(package_mismatch),
    package_mismatch[['ClaimID', 'ClaimedAmount', 'TotalStandardRate', 'min_allowed', 'max_allowed']].head().to_dict()
)

In [79]:
audit_df = pd.DataFrame(audit_log)
audit_df.to_csv("audit_log.csv", index=False)
audit_df.head()


,table,column,issue_type,rule,rows_affected,sample_values
0,Claims_Fact,ClaimID,Duplicate Key,ClaimID must be unique,0,[]
1,Provider_Master,ProviderID,Duplicate Key,ProviderID must be unique,0,[]
2,Member_Master,MemberID,Duplicate Key,MemberID must be unique,0,[]
3,Policy_Master,PolicyID,Duplicate Key,PolicyID must be unique,0,[]
4,Claims_Fact,DischargeDate,Invalid Date Order,DischargeDate < AdmissionDate,0,"{'ClaimID': {}, 'AdmissionDate': {}, 'Discharg..."


In [67]:
claims.to_csv("Claims_Fact_clean.csv", index=False)
providers.to_csv("Provider_Master_clean.csv", index=False)
members.to_csv("Member_Master_clean.csv", index=False)
policies.to_csv("Policy_Master_clean.csv", index=False)


In [68]:
claims.columns

Index(['ClaimID', 'MemberID', 'PolicyID', 'ProviderID', 'ClaimType',
       'ClaimSubmissionDate', 'AdmissionDate', 'DischargeDate',
       'ClaimedAmount', 'ApprovedAmount', 'ICDCode(s)', 'ProcedureCode(s)',
       'PreAuthNumber', 'PreAuthStatus', 'QueryRaiseDate', 'SettlementDate',
       'TPANotes', 'Currency', 'NetworkStatus', 'ProcedureCode',
       'ProcedureCategory', 'StandardRate', 'z_score_claim', 'ICD_list',
       'Procedure_list'],
      dtype='object')

In [69]:
claims["SettlementDate"].isnull().sum()

np.int64(1234)